In [1]:
import wandb
import pytorch_lightning as pl
import torch
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, Resize, ToTensor

In [2]:
wandb.init(project='test')

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: penchekrak (use `wandb login --relogin` to force relogin)


In [3]:
img_size = (512, 512)

def target_transform(target):
    size = float(target['annotation']['size']['width']), float(target['annotation']['size']['height']) 
    label = [float(target['annotation']['object'][0]['bndbox'][coord]) for coord in ['xmin', 'ymin', 'xmax', 'ymax']] 
    label[0] /= size[0]
    label[2] /= size[0]
    label[1] /= size[1]
    label[3] /= size[1]
    return torch.Tensor(label)
dataset = VOCDetection(
    root='../data',
    download=True,
    year='2007',
    image_set='val',
    transform=Compose([Resize(img_size), ToTensor()]),
    target_transform=target_transform
)

Using downloaded and verified file: ../data/VOCtrainval_06-Nov-2007.tar


In [4]:
from torch.utils.data import DataLoader

In [5]:
dataloader = DataLoader(dataset, batch_size=10)

In [7]:
x, y = next(iter(dataloader))

Object detection tutorial

In [10]:
from pytorch_lightning.core.lightning import LightningModule
import torch.nn.functional as F
from torch.optim import Adam
from pytorch_lightning.loggers import WandbLogger

class DummyDetector(LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3, 10, kernel_size=3, padding=1)
        self.act1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(10, 4, kernel_size=512)
        self.flatten = torch.nn.Flatten()
        self.sigm = torch.nn.Sigmoid()
    
    def forward(self, x):
        return self.sigm(self.flatten(self.conv2(self.act1(self.conv1(x)))))
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        outs = self(x)
        loss = F.mse_loss(outs, y)
        self.logger.experiment.log({'image': wandb.Image(
            x[0].permute(1, 2, 0).detach().cpu().numpy(),
            boxes={
    "predictions": {
        "box_data": [{
            "position": {
                "minX": outs[0][0].item(),
                "maxX": outs[0][2].item(),
                "minY": outs[0][1].item(),
                "maxY": outs[0][3].item(),
            },
            "class_id" : 0,
            "box_caption": "box",
            "scores" : {
                "acc": 0,
                "loss": 0
            },
        },
        ],
        "class_labels": {0: 'box'}
    }
        }),
    'train_loss': loss.item()
    })
        return loss
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-3)

In [12]:
from pytorch_lightning import Trainer

model = DummyDetector()
trainer = Trainer(logger=WandbLogger(), gpus=1)
trainer.fit(model, dataloader)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name    | Type    | Params
------------------------------------
0 | conv1   | Conv2d  | 280   
1 | act1    | ReLU    | 0     
2 | conv2   | Conv2d  | 10.5 M
3 | flatten | Flatten | 0     
4 | sigm    | Sigmoid | 0     
------------------------------------
10.5 M    Trainable params
0         Non-trainable params
10.5 M    Total params
41.944    Total estimated model params size (MB)
/data/annispiridonov/research_practice_venv/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/data/annispiridonov/research_practice_venv/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [13]:
model(x)[0]

tensor([0., 0., 1., 1.], grad_fn=<SelectBackward>)

Distillation Tutorial

In [ ]:
# TODO